In [34]:
import pandas as pd
import numpy as np
#import sframe
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import json
from __future__ import division

In [2]:
dtype_dict = {'id':int, 'member_Id':int, 'loan_amnt':float, 'funded_amnt':float, 'funded_amnt_inv':float, 
              'term':str, 'float_rate':float, 'installment':float, 'grade':str, 'sub_grade':str, 'emp_title':str,
              'emp_length':str, 'home_ownership':str, 'annual_inc':float, 'is_inc_v':str, 'issue_d':str, 'loan_status':str, 
              'pymnt_plan':str, 'url':str, 'desc':str, 'purpose':str,'title':str,'zipcode':str,'addr_state':str,'dti':float,'deling_2yrs':float,'earliest_cr_line':str,
             'inq_last_6mths':float,'mths_since_last_delinq':float,'mths_since_last_record':float,'open_acc':float,'pub_rec':float,
             'revol_bal':float,'revol_util':float,'total_acc':float,'initial_list_status':str,'out_prncp':float,'out_prncp_inv':float,
             'total_pymnt':float,'total_pymnt_inv':float,'total_rec_prncp':float,'total_rec_float':float,'total_rec_late_fee':float,
              'recoveries':float,'collection_recovery_fee':float,'last_pymnt_d':str,'last_pymnt_amnt':float,'next_pymnt_d':str,
              'last_credit_pull_d':str,'collections_12_mths_ex_med':float,'mths_since_last_major_derog':float,
              'policy_code':float,'not_compliant':float,'status':str,'inactive_loans':float,'bad_loans':int,'emp_length_num':float,
              'grade_num':float,'sub_grade_num':float,'delinq_2yrs_zero':float,'pub_rec_zero':float,'collections_12_mths_zero':float,
              'short_emp':float,'payment_inc_ratio':float,'final_d':str,'last_delinq_none':float,'last_record_none':float,'last_major_derog_none':float}
#str,float,int

In [3]:
loans = pd.read_csv("/Users/wxu/workspace/ML-UW-Coursera/classification/week3/lending-club-data.csv",header=0,dtype = dtype_dict)

In [4]:
loans["safe_loans"] = loans['bad_loans'].apply(lambda x: 1 if x==0 else -1) #change for itself does not need loc

In [5]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

In [6]:
target = 'safe_loans'

In [7]:
loans = loans[features + [target]]

In [132]:
loans.dtypes

grade                     object
sub_grade                 object
short_emp                float64
emp_length_num           float64
home_ownership            object
dti                      float64
purpose                   object
term                      object
last_delinq_none         float64
last_major_derog_none    float64
revol_util               float64
total_rec_late_fee       float64
safe_loans                 int64
dtype: object

In [8]:
#one-hot encoding for categorical data

categorical_variables = []
for feat_name, feat_type in zip(loans.columns, loans.dtypes):
    if feat_type == object:
        categorical_variables.append(feat_name)
print categorical_variables

['grade', 'sub_grade', 'home_ownership', 'purpose', 'term']


In [9]:
loansWithDummy = pd.get_dummies(loans,columns = categorical_variables)

In [10]:
loans = loansWithDummy.fillna(0)

In [11]:
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week3/trainingIdx.json") as myjson:
    for line in myjson:
        trainIdx = json.loads(line)
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week3/validIdx.json") as myjson:
    for line in myjson:
        validIdx = json.loads(line)
train = loans.iloc[trainIdx]
valid = loans.iloc[validIdx]

In [12]:
print train.shape
print valid.shape

(37224, 68)
(9284, 68)


In [13]:
validation_safe_loans = valid[valid[target] == 1]
validation_risky_loans = valid[valid[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

short_emp  emp_length_num    dti  last_delinq_none  last_major_derog_none  \
19          0              11  11.18                 1                      1   
79          0              10  16.85                 1                      1   
24          0               3  13.97                 0                      1   
41          0              11  16.33                 1                      1   

    revol_util  total_rec_late_fee  safe_loans  grade_A  grade_B  \
19        82.4                   0           1        0        1   
79        96.4                   0           1        0        0   
24        59.5                   0          -1        0        0   
41        62.1                   0          -1        1        0   

         ...         purpose_house  purpose_major_purchase  purpose_medical  \
19       ...                     0                       0                0   
79       ...                     0                       0                0   
24       ...                     0                       0                0   
41       ...                     0                       0                0   

    purpose_moving  purpose_other  purpose_small_business  purpose_vacation  \
19               0              0                       0                 0   
79               0              0                       0                 0   
24               0              1                       0                 0   
41               0              0                       0                 0   

    purpose_wedding  term_ 36 months  term_ 60 months  
19                0                1                0  
79                0                1                0  
24                0                0                1  
41                0                1                0  

[4 rows x 68 columns]

In [14]:
def get_numpy_data(df,features,label):
    # df: a data frame that need to be transformed
    # features: a list of strings for the features name
    # label : single string for the class labels
    features_frame = df[features]
    features_matrix = features_frame.as_matrix()
    label_sarray = df[[label]]
    label_array= label_sarray.as_matrix()
    return(features_matrix,label_array)

In [41]:
featureName = list(loans.columns)
featureName.remove("safe_loans")
feature_matrix_train,label_train = get_numpy_data(train,featureName,"safe_loans")
feature_sample_validation_data,label_sample_validation_data =  get_numpy_data(sample_validation_data,featureName,"safe_loans")
feature_valid_data,label_valid =  get_numpy_data(valid,featureName,"safe_loans")


In [16]:
tree1 = DecisionTreeClassifier(max_depth=6)
tree2 = DecisionTreeClassifier(max_depth=2)

In [17]:
decision_tree_model = tree1.fit(feature_matrix_train,label_train)
small_model = tree2.fit(feature_matrix_train,label_train)
#decision_tree_model = tree1.fit(train_feature,train_label)
#small_model = tree2.fit(train_feature,train_label)

In [18]:
#prediction
pred_big = decision_tree_model.predict(feature_sample_validation_data)
pred_big_prob = decision_tree_model.predict_proba(feature_sample_validation_data)
pred_small = small_model.predict(feature_sample_validation_data)
pred_small_prob = small_model.predict_proba(feature_sample_validation_data)

In [19]:
print pred_big
print pred_small
print label_sample_validation_data

[ 1 -1 -1  1]
[ 1 -1 -1  1]
[[ 1]
 [ 1]
 [-1]
 [-1]]


In [163]:
pred_big_prob

array([[ 0.34156543,  0.65843457],
       [ 0.53630646,  0.46369354],
       [ 0.64750958,  0.35249042],
       [ 0.20789474,  0.79210526]])

In [165]:
pred_small_prob

array([[ 0.41896585,  0.58103415],
       [ 0.59255339,  0.40744661],
       [ 0.59255339,  0.40744661],
       [ 0.23120112,  0.76879888]])

In [46]:
#pred on training
pred_big_train = decision_tree_model.predict(feature_matrix_train)
pred_small_train = small_model.predict(feature_matrix_train)
pred_big_train.shape=(37224,1)
pred_small_train.shape=(37224,1)#compare 2 np array ,check whether their shape are the same
print (pred_big_train==label_train).sum()/label_train.shape[0]
print (pred_small_train==label_train).sum()/label_train.shape[0]
pred_big_valid = decision_tree_model.predict(feature_valid_data)
pred_small_valid = small_model.predict(feature_valid_data)
pred_big_valid.shape=(9284,1)
pred_small_valid.shape=(9284,1)
print (pred_big_valid==label_valid).sum()/label_valid.shape[0]
print (pred_small_valid==label_valid).sum()/label_valid.shape[0]

0.640527616591
0.613502041694
0.636148211978
0.619345109866


In [47]:
tree3 =  DecisionTreeClassifier(max_depth=10)
super_big_model = tree3.fit(feature_matrix_train,label_train)
pred_superBig_train = super_big_model.predict(feature_matrix_train)
pred_superBig_valid = super_big_model.predict(feature_valid_data)
pred_superBig_train .shape=(37224,1)
pred_superBig_train .shape=(37224,1)
pred_superBig_valid=(9284,1)
pred_superBig_valid=(9284,1)
print (pred_superBig_train==label_train).sum()/label_train.shape[0]
print (pred_superBig_train==label_train).sum()/label_train.shape[0]
print (pred_superBig_valid==label_valid).sum()/label_valid.shape[0]
print (pred_superBig_valid==label_valid).sum()/label_valid.shape[0]

0.663845905867
0.663845905867
0.496553209823
0.496553209823


In [60]:
#false neg
a=pred_big_valid== -1 
b=label_valid ==1
c=a&b
c.sum()
#c 6613

1717

In [61]:
#false pos
d=pred_big_valid== 1 
e=label_valid ==-1
f=d&e
f.sum()

1661

In [ ]:
# save int as float if not sure
# change a column by it self or create a new column does not need to use loc.
# pd.series.apply could equal to pd.series.map when using lambda function
# sk learn for categorical, using get_dummies(df,cat_col_name_list)